# Model Evaluation Script

In [124]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras import regularizers, optimizers
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from keras import backend as K
from sklearn.metrics import log_loss


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../March-Madness"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Data
Data_Organization.ipynb
Final
Final_Data_Organization.ipynb
Final MLP.ipynb
GiantKillerInfo.txt
logs
MarchMadnessAdvStats.csv
MarchMadnessFeatureDifferences.csv
MarchMadnessFeatures.csv
MarchMadnessTest.csv
MLP Make Bracket.ipynb
ModelEvaluation.ipynb
OldNbs
README.md
SubmissionData.py
Submissions
tENSORFLOW.ipynb
tENSORFLOW.py
tENSORFLOW-TB.ipynb



## Load pre-organized data

In [125]:
data_dir = '../March-Madness/'
df_features = pd.read_csv(data_dir + 'MarchMadnessFeatures.csv')

Format and scale the data

In [126]:
X = df_features.iloc[:,1:]
xDim = np.shape(X)[1]
X_train = X.values.reshape(-1,xDim)
y_train = df_features.Result.values

print('Feature vector dimension is: %.2f' % xDim)
# print(X[:5, :])

Feature vector dimension is: 34.00


In [127]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_train = pca.fit_transform(X_train)

In [128]:
xDim = np.shape(X_train)[1]

# Load Training Data


In [129]:
df_test = pd.read_csv(data_dir+'MarchMadnessTest.csv')

X_test = df_test.iloc[:,1:]
xDimTest = np.shape(X_test)[1]
X_test = X_test.values.reshape(-1,xDimTest)
y_test = df_test.Result.values

X_test = pca.transform(X_test)

Create Kfold splits

In [130]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

In [131]:
np.random.seed()

## MLP

Various properties

In [132]:
dropRate = 0.3
numBatch = 20
numEpoch = 120
learningRate = 1e-4

Base Single Layer Model

In [139]:
# MLP model
MLP = Sequential()
MLP.name = 'MLP'
MLP.add(Dense(5, input_dim=xDim, kernel_initializer='glorot_normal',activation = 'tanh'))
MLP.add(Dropout(dropRate))
MLP.add(Dense(200,kernel_initializer='glorot_normal',activation = 'tanh'))
MLP.add(Dropout(dropRate))
MLP.add(Dense(200, kernel_initializer='glorot_normal',activation = 'tanh'))
MLP.add(Dropout(dropRate))
MLP.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))

# Compile model
adam = optimizers.Adam(lr=learningRate, amsgrad=True)
MLP.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [140]:
MLP.fit(X_train, y_train, epochs=numEpoch, batch_size=numBatch, verbose=1)

Epoch 1/120
530/530 [==============================] - 1s 1ms/step - loss: 0.7038 - acc: 0.4962
Epoch 2/120
530/530 [==============================] - 0s 57us/step - loss: 0.7023 - acc: 0.5226
Epoch 3/120
530/530 [==============================] - 0s 58us/step - loss: 0.7053 - acc: 0.5000
Epoch 4/120
530/530 [==============================] - 0s 53us/step - loss: 0.7045 - acc: 0.4755
Epoch 5/120
530/530 [==============================] - 0s 61us/step - loss: 0.7101 - acc: 0.4774
Epoch 6/120
530/530 [==============================] - 0s 54us/step - loss: 0.6977 - acc: 0.5151
Epoch 7/120
530/530 [==============================] - 0s 57us/step - loss: 0.7005 - acc: 0.5094
Epoch 8/120
530/530 [==============================] - 0s 55us/step - loss: 0.6941 - acc: 0.5170
Epoch 9/120
530/530 [==============================] - 0s 50us/step - loss: 0.7060 - acc: 0.4792
Epoch 10/120
530/530 [==============================] - 0s 56us/step - loss: 0.6821 - acc: 0.5491
Epoch 11/120
530/530 [========

In [141]:
MLP.evaluate(X_test, y_test)

402/402 [==============================] - 0s 661us/step


[0.6227560594900331, 0.6815920400234005]

## Random Forest

In [ ]:
fore = RandomForestClassifier(criterion='entropy', n_estimators = 50, max_depth = 4, oob_score=True)

fore.fit(X_train, y_train)

y_pred =  fore.predict_proba(X_train)[:,1].reshape(-1,1)
LL = log_loss( y_train, y_pred)
print("Training set score: {:4}" .format(LL))
y_pred =  fore.predict_proba(X_test)[:,1].reshape(-1,1)
LL = log_loss( y_test, y_pred)
print("Validation set score: {:4}".format(LL))

In [120]:
accTest = fore.score(X_test, y_test)
print("Validation accuracy: {:4}".format(accTest))

Validation accuracy: 0.736318407960199


## Making predictions with model

### Extract data desired

In [96]:
data_dir = '../March-Madness/Final/'
df_sample_sub = pd.read_csv(data_dir + 'SampleSubmissionStage2.csv')
data_file = data_dir + 'MarchMadnessAdvStats.csv'
df_adv = pd.read_csv(data_file)
df_seeds = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')


n_test_games = len(df_sample_sub)

def get_year_t1_t2(ID):
    """Return a tuple with ints `year`, `team1` and `team2`."""
    return (int(x) for x in ID.split('_'))

def seed_to_int(seed):
    '''Get just the digits from the seeding. Return as int'''
    s_int = int(seed[1:3])
    return s_int

In [97]:
print('Loading data for submission test')

# Make the seeding an integer
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(columns=['Seed'], inplace=True) # This is the string label
df_seeds.head()


T1_seed = []
T1_adv = []
T2_adv = []
T2_seed = []
for ii, row in df_sample_sub.iterrows():
    year, t1, t2 = get_year_t1_t2(row.ID)
    t1_seed = df_seeds[(df_seeds.TeamID == t1) & (df_seeds.Season == year)].seed_int.values[0]
    t2_seed = df_seeds[(df_seeds.TeamID == t2) & (df_seeds.Season == year)].seed_int.values[0]
    t1_adv = df_adv[(df_adv.TeamID == t1) & (df_adv.Season == year)].values[0]
    t2_adv = df_adv[(df_adv.TeamID == t2) & (df_adv.Season == year)].values[0]
    T1_seed.append(t1_seed)
    T1_adv.append(t1_adv)
    T2_seed.append(t2_seed)
    T2_adv.append(t2_adv)

T1_adv = [row[2:] for row in T1_adv]
T2_adv = [row[2:] for row in T2_adv]
T1_seed = np.reshape(T1_seed, [n_test_games,-1]).tolist()
T2_seed = np.reshape(T2_seed, [n_test_games, -1]).tolist()
X_pred = np.concatenate((T1_seed, T1_adv, T2_seed, T2_adv), axis=1)

df_subData = pd.DataFrame(np.array(X_pred).reshape(np.shape(X_pred)[0], np.shape(X_pred)[1]))

xDim = np.shape(df_subData)[1]
X_pred = df_subData.values.reshape(-1,xDim)

Loading data for submission test


In [98]:
np.shape(X_pred)

(2278, 34)

In [99]:
X_pred = pca.transform(X_pred)

In [100]:
preds = MLP.predict_proba(X_pred)

# df_sample_sub = pd.DataFrame()
# clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = preds
df_sample_sub.shape

(2278, 2)

In [117]:
preds = fore.predict_proba(X_pred)

# df_sample_sub = pd.DataFrame()
# clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = preds
df_sample_sub.shape

(2278, 2)

In [118]:
filename = 'RF'
save_dir = '../March-Madness/Final/'
c=0
ext = '.csv'
if os.path.exists(save_dir+filename+ext):
    while os.path.exists(filename+ext):
        c+=1
    filename = filename+'_'+str(c)
    df_sample_sub.to_csv(save_dir+filename+ext, index=False)
else:
    df_sample_sub.to_csv(save_dir+filename+ext, index=False)